Reload .py files whenever there are changes

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Local imports from this project

In [16]:
from mdf import *
from mdf.ops import *
from mdf.data import *

External imports from dependencies

In [17]:
from pytorch3d.ops.laplacian_matrices import norm_laplacian, cot_laplacian, laplacian
from pytorch3d.ops import norm_laplacian, cot_laplacian, laplacian
from pyvista.plotting.plotter import Plotter
import torch.distributions as distrib
import plotly.graph_objects as go
import plotly.express as px
from torch import Tensor
from typing import cast
import torch_geometric
import torch.nn as nn
import pyvista as pv
import scipy.linalg
import torch.linalg
import numpy as np
import pytorch3d
import einops
import torch

Environment setup

In [18]:
pv.set_jupyter_backend('trame')

Data Loading

In [19]:
# Download standard shapes
data = DataManager(data_dir=data_dir, cache_dir=cache_dir)

How to apply a texture to a 3d mesh

In [20]:
vista  = data.objects['stanford-bunny'].vista
texture = pv.Texture(data.weather[42]['data'])
vista.texture_map_to_sphere(inplace=True)
vista.plot(texture=texture)

/home/invokariman/.cache/pypoetry/virtualenvs/mdf-B-9uJKhD-py3.11/lib/python3.11/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


Widget(value='<iframe src="http://localhost:41897/index.html?ui=P_0x7ee9911d6bd0_6&reconnect=auto" class="pyvi…

Compute positional embedding using Laplace-Beltrami Operator

In [28]:
lbo = LBOEmbedding.from_poly(vista, k_evecs=10, device=torch.device('cuda'))

In [33]:
mesh = pv.PolyData(lbo.verts.numpy(), lbo.faces.numpy())
p = Plotter()
evecs = lbo(torch.arange(lbo.verts.size(0)))
p.add_mesh(mesh, scalars=evecs[:, 1:2].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.show()

Widget(value='<iframe src="http://localhost:41897/index.html?ui=P_0x7ee8d13c9650_16&reconnect=auto" class="pyv…

In [46]:
p = Plotter()
m_sampler = MeshSampler(verts=lbo.verts, faces=lbo.faces)
pts, verts, coefs = m_sampler.sample(10000)
evecs = lbo(verts, coefs)
mesh = pv.PolyData(pts.numpy())
p.add_mesh(mesh, scalars=evecs[:, 1:2].sum(dim=1), clim=[-1, 1], cmap='RdBu_r', show_edges=True)
p.show()

Widget(value='<iframe src="http://localhost:41897/index.html?ui=P_0x7ee8d5840b90_29&reconnect=auto" class="pyv…